Segue abaixo o nome, a abreviação e a descrição das 21 colunas de informações apresentadas
nos resultados de uma pesquisa no Léxico do Português Brasileiro, versão Alfa.
Ortografia (orto): forma ortográfica da palavra em letras minúsculas (com exceção dos
nomes próprios), respeitando os acentos específicos de cada palavra37
.
- Categoria gramatical (cat_gram):<
    - categorial gramatical da palavra (adj, adv, gram, nom,num, prop, ver).

- Informação gramatical (inf_gram): 
    - informações gramaticais sobre a palavra (ex. singular/plural, masculino/feminino, passado/presente/futuro, 1/2/3 pessoas, etc.). 
- Frequência ortográfica (freq_orto): 
    - número de vezes que a palavra aparece no NILC (cerca de 32 milhões de palavras).

- Frequência ortográfica por milhão (freq_orto/M): 
     - número de vezes que a palavra aparece entre 1 milhão de palavras. Valor padrão para frequência de palavras.

- Logaritmo natural da frequência ortográfica (log10_freq_orto): 
    - logarítmico natural da frequência ortográfica. Os valores logarítmicos são utilizados para linearizar-se o comportamento das frequências das palavras no corpus.

- Número de letras (nb_letras):
    - número de letras da palavra.

- Número de homógrafas (nb_homogr):
    - número de palavras homógrafas. Palavras que possuem a mesma ortografia ou diferenças de acentos, mas pertencem a categorias gramaticais diferentes.

- Homógrafas (homografas): 
     - categorias gramaticais das palavras homógrafas.

- Ponto de unicidade ortográfico (pu_orto): 
    - letra a partir da qual a palavra se dissocia das outras, ou seja, letra a partir da qual a palavra é única. Sentido da esquerda para direita.

- Vizinhos ortográficos (viz_orto): 
     - número de vizinhos ortográficos a partir do N de Coltheart, ou seja, alterando-se apenas uma letra por vez (Coltheart et al., 1977).

- Distância de Leveinshtein ortográfica (old20):
     - distância ortográfica de Leveinshtein das 20 palavras mais póximas calculadas a partir de regressões lineares (Yarkoni et al., 2008).

- Estrutura CVCV (CVCV_orto):
    - estrutura CVCV da palavra, onde consoantes são C e vogais são V. Ainda, A para acentos, P para pontuação, N para números e S para símbolos.

- Bigramas (bigramas): 
    - bigramas que constituem a palavra separados por “_” e limitados por “#”. O número de bigramas é igual ao número de letras da palavra mais 1.

- Trigramas (trigramas):
     - trigramas que constituem a palavra separados por “_” e limitados por “#”. O número de trigramas é igual ao número de letras da palavra.

- Ortografia invertida (inv_orto):
    - forma invertida da ortografia (orto).

- Estrutura CVCV invertida (inv_CVCV_orto):
    - estrutura CVCV da palavra invertida a partir de (CVCV_orto).

- Bigramas invertidos (inv_bigra): 
     - bigramas que constituem a palavra separados por “_” e limitados por “#” invertidos a partir de (bigramas).

- Trigramas invertidos (inv_trigra):
    - trigramas que constituem a palavra separados por “_” e limitados por “#” invertidos a partir de (trigramas).

- Número aleatório entre 0 e 1 (aleatorio): 
    - número aleatório entre 0 e 1 com oito algarismos de precisão.

- Número de identificação (id): 
    - número de identificação da palavra designado a partir da organização do corpus por frequência decrescente e ordem alfabética a-z. O número de identificação é a posição da palavra no corpus e no léxico


In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import datetime
import json

In [ ]:
def verifica_palavrao(x):
    if x not in palavroes:
        return x
    elif x not in norm_palavroes:
        return x

In [ ]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
FILE_NAME = 'lexporbr_alfa_excel.xlsx'

In [ ]:
df_lexico = pd.read_excel(FILE_NAME)

In [ ]:
#lendo o ARQUIVO DE PALAVROES 
f = open('Words/palavroes.txt') 
palavroes = f.readlines()
f.close()

In [ ]:
palavroes = pd.Series(palavroes).apply(lambda x: x.lower().replace('\n ', '').replace(' ', ''))
norm_palavroes = pd.Series(palavroes).apply(lambda row: strip_accents(row))

In [ ]:
lexico_filtered = df_lexico[(df_lexico['nb_letras'] > 2) & (df_lexico['nb_letras'] <= 6) & (df_lexico['cat_gram'] == 'nom')].sort_values(by='freq_orto/M', ascending=False)

### Verifica e retira palavras escolhidas

In [ ]:
lexico_filtered.ortografia = lexico_filtered['ortografia'].apply(lambda row: verifica_palavrao(row))

### tratar toda a base para tirar palavras com hifen, ponto, caracteres especiais, numeros

In [ ]:
def tratativa(X):  
    '''
    Entrada: series
    X_norm 
        - retira acentos das palavras 
 
    list comprehension
        - remove palavras com hifen
        - remove "."
        - remove palavra com menos de 2 caracter
        - remove palavras com letras maisculas (nomes proprios)
        - remove palavras com caracteres especiais utilizando regex
        - retira palavras escolhidas
        
    return: 2 listas, 1 de palavras sem acentos e outra normal
    '''
    
    #verificacao do regex para caracteres especiais
    string_check = re.compile("['-@_!#$%^&*()<>?/\|}{~:¹²³£¢¬ªº°]")

    word_list = [x for x in X
                if  '-' not in x 
                and string_check.search(x) == None
                and len(x) > 2 
                and '.' not in x 
                and x.lower() == x
                ]
    
    
    # removendo acentos
    y = pd.Series(word_list)
    X_norm = y.apply(lambda row: strip_accents(row))

    print("len lista palavras normalizadas: ", len(X_norm))
    print("len lista palavras : ", len(word_list))

    
    return X_norm.to_list(), word_list

In [ ]:
normalized_word_list, word_list= tratativa(lexico_filtered.head(1200)['ortografia'])

### Criando Dataframe com os puzzles diarios


In [ ]:
def df_puzzles(normalized_word_list, word_list):
    
    a = datetime.datetime(2022, 3, 11)

    numdays = 365
    dateList = []
    for x in range (0, numdays):
        dateList.append(a + datetime.timedelta(days = x))


    dic = {'norm_word': normalized_word_list,
           'word': word_list}

    word_df = pd.DataFrame(dic, columns=['norm_word', 'word'])



    df_puzzle_day = pd.DataFrame(dateList, columns=['date']) # cria um dataframe com todas as datas para agrupamento

    # cria colunas que serão utilizadas no dataframe
    df_puzzle_day['day_puzzle'] = 0
    df_puzzle_day['day_puzzle_norm'] =  0
    df_puzzle_day['letters'] = 0
    df_puzzle_day['letters_norm'] = 0
    df_puzzle_day['word_letters_list'] = 0
    df_puzzle_day['word_letters_list_norm'] = 0


    #loop que varre todas as datas enumerando-as
    for index, date in enumerate(dateList):
        sample = word_df.sample(3, random_state=index) # pega o sample da base de lista utilizando como random state o index do df, para futuras consultas
        day_puzzle = sample.word.tolist() # transforma as 3 palavras selecionadas do sample em uma lista
        day_puzzle_norm = sample.norm_word.tolist() # transforma as 3 palavras selecionadas do sample em uma lista


        # Seleciona as letras das palavras selecionadas em uma listas 3 (uma para cada palavra)
        day_puzzle_letters = []
        for word in day_puzzle:
            day_puzzle_letters.append(sorted(word))

        # transforma a lista de listas em uma unica lista, ordendando as letras
        flat_letter_list = sorted([item for sublist in day_puzzle_letters for item in sublist])

        day_puzzle_letters_norm = []
        for word in day_puzzle_norm:
            day_puzzle_letters_norm.append(sorted(word))

        # transforma a lista de listas em uma unica lista, ordendando as letras
        flat_letter_list_norm = sorted([item for sublist in day_puzzle_letters_norm for item in sublist])


        #atribui os dados obtidos no looping para o dataframe
        df_puzzle_day['date'][index] = date.date()
        df_puzzle_day['day_puzzle'][index] =  day_puzzle
        df_puzzle_day['day_puzzle_norm'][index] =  day_puzzle_norm
        df_puzzle_day['letters'][index] = flat_letter_list
        df_puzzle_day['letters_norm'][index] = flat_letter_list_norm
        df_puzzle_day['word_letters_list'][index] = day_puzzle_letters
        df_puzzle_day['word_letters_list_norm'][index] = day_puzzle_letters_norm

        
    return df_puzzle_day

In [ ]:
df_puzzle_day = df_puzzles(normalized_word_list, word_list)

## CRIANDO O JSON

In [ ]:

i = 0
dic_aux = {}
list = []


for date, letter_norm, solution in zip(df_puzzle_day.date, df_puzzle_day.letters_norm, df_puzzle_day.day_puzzle):
    dic_aux = {
        'id': i,
        'date':str(date),
        'letters': letter_norm,
        'solution': solution
    }    
    list.append(dic_aux)
    i = i+1
    
# #salvando o arquivo json
# with open('Puzzles/daily_puzzles_6.json', 'w') as f:
#     json.dump(list, f)

In [ ]:
#lendo o JSON 
f = open('Puzzles/daily_puzzles_6.json') 
daily_puzzles = json.load(f)
f.close()

In [ ]:
daily_puzzles